In [1]:
!pip install ultralytics roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 19.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 131.4 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


In [2]:
from roboflow import Roboflow
from ultralytics import YOLO
import os
import shutil
import yaml
from glob import glob

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
def remap_and_copy(source_path, dest_path_images, dest_path_labels, target_class_id):
    """
    Copies images and creates remapped label files.
    """
    os.makedirs(dest_path_images, exist_ok=True)
    os.makedirs(dest_path_labels, exist_ok=True)

    # Process images and corresponding labels
    # YOLOv8 datasets usually have 'images' and 'labels' folders
    
    # We look for images in the source
    source_images = glob(os.path.join(source_path, 'images', '*'))
    
    for img_path in source_images:
        # Copy Image
        shutil.copy(img_path, dest_path_images)
        
        # Process Label
        basename = os.path.basename(img_path)
        name_root, _ = os.path.splitext(basename)
        label_file = os.path.join(source_path, 'labels', f"{name_root}.txt")
        
        if os.path.exists(label_file):
            with open(label_file, 'r') as f:
                lines = f.readlines()
            
            new_lines = []
            for line in lines:
                parts = line.strip().split()
                if len(parts) >= 5:
                    # Replace original class_id with target_class_id
                    # We keep coordinates/dimensions same
                    new_line = f"{target_class_id} " + " ".join(parts[1:])
                    new_lines.append(new_line)
            
            if new_lines:
                with open(os.path.join(dest_path_labels, f"{name_root}.txt"), 'w') as f:
                    f.write('\n'.join(new_lines))

In [4]:
api_key = "u0leKBS1HrAOJli1hvLI"

if api_key == "YOUR_API_KEY":
    print("Please replace 'YOUR_API_KEY' within the script.")

# MAPPING CONFIGURATION
# (Workspace, Project, Version, Target_Class_ID, Target_Class_Name)
dataset_config = [
    # GLOBAL CLASS 0: Wagon
    ("wagoncounting", "wagon-dataset", 1, 0, "Wagon"), 
    ("aispry-ob85t", "wagon-detection-zsnyn", 2, 0, "Wagon"),
    ("alisha-nyb7f", "wagon-detection-qxlxh", 1, 0, "Wagon"),
    ("wagons-thdfd", "cv-alt", 2, 0, "Wagon"),
    
    # GLOBAL CLASS 1: Wagon parts
    ("db-rail", "train-wagon-cv-project", 3, 1, "Wagon parts"),
    
    # GLOBAL CLASS 2: Wagon numbers
    ("sedykh-marat-dxrw3", "wagon-numbers-detection", 1, 2, "Wagon numbers"),
    ("student-ih3dc", "wagon-detection-qc7bh", 1, 2, "Wagon numbers"),
]

In [5]:
rf = Roboflow(api_key=api_key)

# Create Merged Dataset Structure
MERGED_DIR = "railway_hackathon_merged"
for split in ['train', 'valid', 'test']:
    os.makedirs(os.path.join(MERGED_DIR, split, 'images'), exist_ok=True)
    os.makedirs(os.path.join(MERGED_DIR, split, 'labels'), exist_ok=True)

print("-" * 60)
print("STEP 1: Downloading & Merging Datasets")
print("-" * 60)

for workspace, project_id, version, target_id, target_name in dataset_config:
    try:
        print(f"Processing {workspace}/{project_id} v{version} -> Class {target_id} ({target_name})")
        project = rf.workspace(workspace).project(project_id)
        dataset = project.version(version).download("yolov8")
        
        location = dataset.location
        
        # Merge Train, Valid, Test splits
        for split in ['train', 'valid', 'test']:
            # Some datasets might use 'train' or 'valid' folders differently, standardizing here
            src_split_path = os.path.join(location, split)
            if not os.path.exists(src_split_path): 
                # Try fallback if Roboflow structure varies
                continue
                
            dest_images = os.path.join(MERGED_DIR, split, 'images')
            dest_labels = os.path.join(MERGED_DIR, split, 'labels')
            
            remap_and_copy(src_split_path, dest_images, dest_labels, target_id)
            
    except Exception as e:
        print(f"Skipping {project_id}: {e}")

------------------------------------------------------------
STEP 1: Downloading & Merging Datasets
------------------------------------------------------------
Processing wagoncounting/wagon-dataset v1 -> Class 0 (Wagon)
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to wagon-dataset-1 in yolov8:: 100%|██████████| 1175/1175 [00:00<00:00, 10003.77it/s]


Processing aispry-ob85t/wagon-detection-zsnyn v2 -> Class 0 (Wagon)
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Wagon-detection-2 in yolov8:: 100%|██████████| 4454/4454 [00:00<00:00, 6333.86it/s]


Processing alisha-nyb7f/wagon-detection-qxlxh v1 -> Class 0 (Wagon)
loading Roboflow workspace...
loading Roboflow project...


...


Extracting Dataset Version Zip to Train-Wagon-CV-Project-3 in yolov8:: 100%|██████████| 5442/5442 [00:00<00:00, 5653.93it/s]


Processing sedykh-marat-dxrw3/wagon-numbers-detection v1 -> Class 2 (Wagon numbers)
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Wagon-numbers-detection-1 in yolov8:: 100%|██████████| 376/376 [00:00<00:00, 7334.34it/s]

Processing student-ih3dc/wagon-detection-qc7bh v1 -> Class 2 (Wagon numbers)
loading Roboflow workspace...


loading Roboflow project...


In [6]:
# Create Custom data.yaml
yaml_content = {
    'train': os.path.abspath(os.path.join(MERGED_DIR, 'train', 'images')),
    'val': os.path.abspath(os.path.join(MERGED_DIR, 'valid', 'images')),
    'test': os.path.abspath(os.path.join(MERGED_DIR, 'test', 'images')),
    'nc': 3,
    'names': ['Wagon', 'Wagon parts', 'Wagon numbers']
}

yaml_path = os.path.join(MERGED_DIR, 'data.yaml')
with open(yaml_path, 'w') as f:
    yaml.dump(yaml_content, f)

print("-" * 60)
print(f"Dataset Merged at: {MERGED_DIR}")

------------------------------------------------------------
Dataset Merged at: railway_hackathon_merged


# Train YOLO Model
Initialize the YOLO model (e.g., `yolov8n.pt`) and execute the training process using the generated `data.yaml` file with specified hyperparameters (epochs, batch size, etc.).

In [7]:
print("STEP 2: Starting Training on Merged Dataset")
print("-" * 60)

# Train
model = YOLO('yolov8s.pt') 

try:
    results = model.train(
        data=yaml_path,
        epochs=50,
        imgsz=640,
        batch=-1,
        project='railway_hackathon_take4',
        name='merged_model_v3',
        freeze=10,
        lr0=0.001,
        patience=10
    )
    print("Training Complete!")
except Exception as e:
    print(f"Training Failed: {e}")

STEP 2: Starting Training on Merged Dataset
------------------------------------------------------------
Ultralytics 8.3.240 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=railway_hackathon_merged/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=merged_model_v3, nbs=64, nms

In [8]:
import os
from google.colab import files
import shutil

# Path based on your logs
weights_path = '/content/railway_hackathon_take4/merged_model_v3/weights/best.pt'
results_folder = '/content/railway_hackathon_take4/merged_model_v3'

if os.path.exists(weights_path):
    print(f"✅ Model found at: {weights_path}")
    
    # Option 1: Download just the weights (faster)
    print("Downloading best.pt...")
    files.download(weights_path)
    
    # Option 2: Zip the full results (logs, graphs, weights) and download
    print("Zipping full results folder...")
    shutil.make_archive('training_results', 'zip', results_folder)
    files.download('training_results.zip')
    
else:
    print("❌ File not found. If the Runtime completely disconnected/reset, the files might be lost.")

✅ Model found at: /content/railway_hackathon_take4/merged_model_v3/weights/best.pt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Zipping full results folder...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
!ls -R /content/railway_hackathon_take4

/content/railway_hackathon_take4:
merged_model_v3

/content/railway_hackathon_take4/merged_model_v3:
args.yaml			 labels.jpg		val_batch0_pred.jpg
BoxF1_curve.png			 results.csv		val_batch1_labels.jpg
BoxP_curve.png			 results.png		val_batch1_pred.jpg
BoxPR_curve.png			 train_batch0.jpg	val_batch2_labels.jpg
BoxR_curve.png			 train_batch1.jpg	val_batch2_pred.jpg
confusion_matrix_normalized.png  train_batch2.jpg	weights
confusion_matrix.png		 val_batch0_labels.jpg

/content/railway_hackathon_take4/merged_model_v3/weights:
best.pt  last.pt


In [11]:
from google.colab import files
import os

source_folder = '/content/railway_hackathon_take4'  # Change if your model folder has a different name/path

if not os.path.exists(source_folder):
    print(f"❌ Folder not found: {source_folder}")
    print("   Check the exact path with !ls /content/")
else:
    print(f"📁 Found folder: {source_folder}")
    print("📦 Zipping the folder...")
    
    zip_path = '/content/railway_hackathon_results.zip'
    
    # Use system zip for speed and reliability
    !zip -r {zip_path} {source_folder}
    
    if os.path.exists(zip_path):
        size_mb = os.path.getsize(zip_path) / (1024 * 1024)
        print(f"✅ Zip created: {zip_path} ({size_mb:.1f} MB)")
        print("⬇️ Starting automatic download to your computer...")
        files.download(zip_path)  # This triggers a direct browser download
    else:
        print("❌ Zip creation failed.")

📁 Found folder: /content/railway_hackathon_take4
📦 Zipping the folder...
  adding: content/railway_hackathon_take4/ (stored 0%)
  adding: content/railway_hackathon_take4/merged_model_v3/ (stored 0%)
  adding: content/railway_hackathon_take4/merged_model_v3/val_batch2_labels.jpg (deflated 16%)
  adding: content/railway_hackathon_take4/merged_model_v3/val_batch0_pred.jpg (deflated 6%)
  adding: content/railway_hackathon_take4/merged_model_v3/train_batch1.jpg (deflated 4%)
  adding: content/railway_hackathon_take4/merged_model_v3/results.png (deflated 7%)
  adding: content/railway_hackathon_take4/merged_model_v3/val_batch0_labels.jpg (deflated 6%)
  adding: content/railway_hackathon_take4/merged_model_v3/BoxF1_curve.png (deflated 9%)
  adding: content/railway_hackathon_take4/merged_model_v3/weights/ (stored 0%)
  adding: content/railway_hackathon_take4/merged_model_v3/weights/best.pt (deflated 8%)
  adding: content/railway_hackathon_take4/merged_model_v3/weights/last.pt (deflated 8%)
  ad

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
!pip install -q google-auth-oauthlib  # One-time if needed

from google.colab import auth
auth.authenticate_user()

import os
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

service = build('drive', 'v3')

# Manual mount simulation
mount_point = '/content/drive'
if not os.path.exists(mount_point):
    os.makedirs(mount_point)

# Copy your files/zip to a temp spot first (if not already zipped)
source_zip = '/content/railway_hackathon_results.zip'  # Or recreate: !zip -r {source_zip} /content/railway_hackathon_take4

# Upload to Drive root (or change folder_id for a subfolder)
file_metadata = {'name': 'railway_hackathon_results.zip'}
media = MediaFileUpload(source_zip, resumable=True)
uploaded_file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()

print(f"✅ Uploaded to Drive! File ID: {uploaded_file.get('id')}")
print("Go to drive.google.com, search for 'railway_hackathon_results.zip', right-click → Download.")

✅ Uploaded to Drive! File ID: 1HXlUkkR2xDc2t7QNtXEgnpbOLmFM7OlD
Go to drive.google.com, search for 'railway_hackathon_results.zip', right-click → Download.
